In [1]:
!pip install -e /Users/rivajangra/Desktop/OQD/TrICal

zsh:1: /usr/local/bin/pip: bad interpreter: /System/Library/Frameworks/Python.framework/Versions/2.7/Resources/Python.app/Contents/MacOS/Python: no such file or directory
Obtaining file:///Users/rivajangra/Desktop/OQD/TrICal
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for trical (pyproject.toml) ... done
  Created wheel for trical: filename=trical-1.0.0-0.editable-py3-none-any.whl size=1340 sha256=b14116a07f653d55f2fa1d56a6a64800e354974ba21c1267f8656eb2cec58372
  Stored in directory: /private/var/folders/6w/8q7qyw2x13q4ycqg4rcv93mw0000gn/T/pip-ephem-wheel-cache-49u_f3vq/wheels/51/c7/61/5ddd5be2d4e0ee8cb6f4b7d30e01b204aa884b33812ba16fc6
Successfully built trical
  Attempting uninstall: trical
    Found existing installation: trical 1.0.0
    Uninstalling trical-1.0.0:
      Successfully uninstalled trical-

In [3]:
from trical.light_matter import utilities
from trical.backend.qutip.rule import time_evolve

ModuleNotFoundError: No module named 'trical.backend.qutip.rule'

In [8]:
from math import sqrt
import numpy as np
from typing import Dict
import oqd_compiler_infrastructure as oqd
from trical.light_matter import utilities
from trical.backend.qutip.passes import time_evolve
from trical.light_matter.interface.Species import Ca40
from trical.light_matter.interface.Chamber import Chamber, Chain, Ion, Laser
from trical.light_matter.interface.structures import VibrationalMode

# from trical.light_matter.interface.structures import Chain as StructureChain 
#from trical.light_matter.interface.structures import Laser as StructureLaser
from trical.light_matter.compiler.rule.codegen import construct_H_tree
from trical.light_matter.compiler.rule.rewrite.laser_parameters import SetLaserWavelengthFromTransition, SetLaserIntensityFromPiTime

# from trical.light_matter.compiler.passes.passManager import PassManager
# from trical.light_matter.compiler.passes.laser_parameters_pass import LaserParametersPass
#from trical.light_matter.compiler.passes.rabi_frequencies_pass import RabiFrequenciesPass

ion = Ca40([("S1/2", -1/2, "g"), ("S1/2", 1/2, "e"), ("P1/2", 1/2, "3")])

laser1 = Laser(
    phase=np.pi / 2,
    wavevector=(np.array([1, 0, 1]) / sqrt(2)).tolist(),
    polarization=(np.array([1, 1, 0]) / sqrt(2)).tolist()
)

laser2 = Laser(
    phase=np.pi,
    wavevector=(np.array([1, 0, 1]) / sqrt(2)).tolist(),
    polarization=[0, 0, 1]
)

chain = Chain(
    ions=[ion],
    trap_freqs=[3e6, 3e6, 1e6],
    selected_modes=[VibrationalMode(eigenfreq=1,eigenvect=[1,1,1],axis=[1,0,0],N=1)]
)

mode = chain.selected_modes[-1]
mode.modecutoff(2)
chain.selected_modes = [mode]

chamber = Chamber(
    chain=chain,
    lasers=[laser1],
    B=3.5928e-4,
    Bhat=(np.array([-1, 0, 1]) / sqrt(2)).tolist()
)

# print(chamber)


SetLaserWavelengthFromTransition(0, ion.transition(("g", "3")))
SetLaserWavelengthFromTransition(1, ion.transition(("e", "3")))

Delta = -180 * 1e9
laser1.detuning = Delta
# delta = 0
# laser2.detune(Delta - delta)

# ns = 1e-9
# pt_1 = 1.67 * ns
# pt_2 = 1.67 * ns

# SetLaserIntensityFromPiTime(0, pt_1, ion.transition(("g", "3")))
# SetLaserIntensityFromPiTime(1, pt_2, ion.transition(("e", "3")))

# if Delta:
#     raman_rabi = (np.pi**2)/(2*abs(Delta)*2*np.pi*pt_1*pt_2)
#     pi_time_tot = np.pi/raman_rabi
#     print("Expected Raman pi time:", pi_time_tot/1e-6 ,  "us")

# timescale = 1e-6

# H = construct_H_tree(chamber, timescale)


printer = oqd.Post(oqd.PrettyPrint())

print(printer(chamber))


Chamber
  - chain: Chain
    - ions: list
      - 0: Ca40
        - mass: float(6.635943757345042e-26)
        - charge: float(1.0)
        - levels: dict
          - g: Level
            - principal: int(4)
            - spin: NoneType(None)
            - orbital: float(0.0)
            - nuclear: float(0.0)
            - spin_orbital: float(0.5)
            - spin_orbital_nuclear: float(0.5)
            - spin_orbital_nuclear_magnetization: float(-0.5)
            - energy: float(0.0)
          - e: Level
            - principal: int(4)
            - spin: NoneType(None)
            - orbital: float(0.0)
            - nuclear: float(0.0)
            - spin_orbital: float(0.5)
            - spin_orbital_nuclear: float(0.5)
            - spin_orbital_nuclear_magnetization: float(0.5)
            - energy: float(0.0)
          - 3: Level
            - principal: int(4)
            - spin: NoneType(None)
            - orbital: float(1.0)
            - nuclear: float(0.0)
            - sp

In [ ]:
import trical.light_matter.compiler.passes.passManager as pm
print(dir(pm))



['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__']


In [ ]:
from qutip import tensor
import numpy as np

duration = 10

times = np.linspace(0, duration, 1000)

psi_ion = ion.state['g']
psi_mot = mode.setstate(2)

psi_0 = tensor(psi_ion, psi_mot)

op1 = chain.ion_projector(1, "g") # project first ion onto "S-" level

op2 = chain.ion_projector(1, "e") # project first ion onto "S+" level

op3 = chain.ion_projector(1, "3") # project first ion onto "P" level

results = time_evolve(H, psi_0, times, expt_ops=[op1, op2, op3], progress_bar= True)
results.quick_plot()

ModuleNotFoundError: No module named 'Species'